<a href="https://colab.research.google.com/github/eers501/eers501.github.io/blob/main/Copy_of_SharedCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown # Hyper-parameter Tuning

# Define some global consts
# Dropout rates in the network
DROPOUT_RATE_CONV = 0 #@param{type:"slider", min:0.0, max:1.0, step:0.05}
DROPOUT_RATE_DENSE = 0.6 #@param{type:"slider", min:0.0, max:1.0, step:0.05}

# Learning rate
LEARNING_RATE = 0.0002 #@param{type:"slider", min:0.0001, max:0.001, step: 0.0001}

# The number of batches
BATCH_SIZE = 2048 #@param{type:"slider", min:256, max:2048, step:256}

# Number of epochs to run
EPOCHS = 80 #@param{type:"slider", min:0, max:300, step:10}

END_CALLBACK = "Iterative Improvement" #@param ["Iterative Improvement", "Target Accuracy", "None"]
DESIRED_ACCURACY = 0.9 #@param{type:"slider", min:0.8, max: 0.98, step:0.01}

# If it hasn't hit the desired accuracy after 1000 runs, it probably never will
if END_CALLBACK != "None":
  EPOCHS = 1000

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print("Tensorflow version %s" % tf.__version__)

Tensorflow version 2.4.1


In [ ]:
# GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Apr 30 14:55:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    37W / 300W |  12205MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Load the dataset
cifar10 = tf.keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [ ]:
# Normalise the image values 0.0-1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Augment the training data using a Keras ImageDataGenerator
data_generator = ImageDataGenerator(
    rotation_range = 15,
    horizontal_flip = True,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rescale=None,
    fill_mode="nearest"
)

# Fit the data generator to the provided images (pre-computes transforms)
data_generator.fit(train_images)

In [ ]:
# Create the testing dataset
testing_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(BATCH_SIZE)

In [ ]:
# Define the model architecture
model = Sequential([
    Conv2D(64, 3, activation='relu', input_shape=(32, 32, 3), padding='same'),
    Conv2D(64, 3, activation='relu', padding='same'),
    MaxPooling2D(),
    Dropout(DROPOUT_RATE_CONV),
    BatchNormalization(),

    Conv2D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, 3, activation='relu', padding='same'),
    MaxPooling2D(),
    Dropout(DROPOUT_RATE_CONV),
    BatchNormalization(),

    Conv2D(256, 3, activation='relu', padding='same'),
    Dropout(DROPOUT_RATE_CONV), # 0.25
    BatchNormalization(),
    Conv2D(256, 3, activation='relu', padding='same'),
    Dropout(DROPOUT_RATE_CONV), # 0.25
    MaxPooling2D(),
    BatchNormalization(),

    Conv2D(512, 3, activation='relu', padding='same'),
    Dropout(DROPOUT_RATE_CONV), # 0.5
    BatchNormalization(),
    Conv2D(512, 3, activation='relu', padding='same'),
    Dropout(DROPOUT_RATE_CONV), # 0.6
    MaxPooling2D(),
    BatchNormalization(),

    Flatten(),
    Dense(2048, activation='relu'),
    BatchNormalization(),
    Dropout(DROPOUT_RATE_DENSE),
    Dense(10, activation=tf.keras.activations.softmax)
])

In [ ]:
# Compile the model with the AMSGrad variant of the Adam optimiser,
# using Sparse Categorical Crossentropy as a loss function
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, amsgrad=True),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy("accuracy")])

In [ ]:
def save_model(model):
  SAVE_TO_DRIVE = True #@param ["True", "False"] {type:"raw"}

  filepath = ""

  if (SAVE_TO_DRIVE):
    # Save the model to Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    filepath = "drive/MyDrive/tf-models/cifar10-classifier-"
    filepath += str(int(test_acc * 100))
  else:
    filepath = "tf-models/cifar10-classifier-"
    filepath += str(int(test_acc * 100))

  model.save(filepath)

In [ ]:
# Train the model on the training data
callbacks=None

next_target_accuracy = DESIRED_ACCURACY

if END_CALLBACK == "Target Accuracy":
  print("Using target accuracy of %f\%%" % (DESIRED_ACCURACY * 100.0))
  class AccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if (logs.get('val_accuracy') > DESIRED_ACCURACY):
        print("Stopping training as desired accuracy has been reached")
        self.model.stop_training = True

  callbacks = AccuracyCallback() 

elif END_CALLBACK == "Iterative Improvement":
  print("Using iterative improvement.  Initial target: %f%%" % (next_target_accuracy * 100.0))
  class IterativeImprovementCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      global next_target_accuracy
      if (logs.get('val_accuracy') > next_target_accuracy):
        print("Reached next milestone %f.  Saving and incrementing target by 1." % (next_target_accuracy))
        next_target_accuracy += 1.0
        save_model(self.model)

  callbacks = IterativeImprovementCallback()

else:
  print("Not using callbacks")
  class NoneCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        pass

  callbacks = NoneCallback()
  

history = model.fit(
    x=data_generator.flow(train_images, train_labels, BATCH_SIZE)
    , epochs=EPOCHS
    , steps_per_epoch=len(train_images)/BATCH_SIZE
    , validation_data=testing_dataset
    , callbacks=[callbacks]
)

Using iterative improvement.  Initial target: 90.000000%
Epoch 1/1000
24/24 [==============================] - 23s 901ms/step - loss: 3.1137 - accuracy: 0.2519 - val_loss: 2.3315 - val_accuracy: 0.1000
Epoch 2/1000
24/24 [==============================] - 22s 896ms/step - loss: 2.2233 - accuracy: 0.3763 - val_loss: 2.4174 - val_accuracy: 0.1000
Epoch 3/1000
24/24 [==============================] - 22s 908ms/step - loss: 1.8978 - accuracy: 0.4321 - val_loss: 2.5516 - val_accuracy: 0.1000
Epoch 4/1000
24/24 [==============================] - 22s 896ms/step - loss: 1.7157 - accuracy: 0.4682 - val_loss: 2.7388 - val_accuracy: 0.1001
Epoch 5/1000
24/24 [==============================] - 22s 888ms/step - loss: 1.5754 - accuracy: 0.4987 - val_loss: 3.1105 - val_accuracy: 0.1010
Epoch 6/1000
24/24 [==============================] - 22s 894ms/step - loss: 1.4473 - accuracy: 0.5303 - val_loss: 3.3744 - val_accuracy: 0.1002
Epoch 7/1000
24/24 [==============================] - 22s 884ms/step - lo

In [ ]:
# Test model accuracy on test set
test_loss, test_acc = model.evaluate(testing_dataset)

print('\nTest accuracy:', test_acc)

In [ ]:
# Save the final model if we haven't already been doing so
if END_CALLBACK != "Iterative Improvement":
  save_model(model)

In [ ]:
# Plot accuracy against epochs
plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  

In [ ]:
# Plot loss against epochs
plt.subplot(212)  
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()